# Assignment 02

- Name: Ran Arino
- Student ID: 153073200
- Email: rarino@myseneca.ca
- Course: Machine Learning
- Course ID: BDA500NAA.05380.2237
- Professor: Dr. Amir Moslemi

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from keras.applications.vgg16 import VGG16, preprocess_input
from keras.models import Model, Sequential
from keras.layers import Dense, Flatten, Input
from sklearn.model_selection import train_test_split

In [2]:
# load dataset
from mat4py import loadmat 
data = loadmat('CatDog.mat')

In [3]:
# get the feature matrix
X = np.array(data['X'])
print(f"Observations: {X.shape[0]}, Features: {X.shape[1]}")

# get the target vector
y = np.array(data['G'])
print(f'Dimension: {y.shape}')

Observations: 4096, Features: 242
Dimension: (242,)


In [4]:
# convert feature matrix so that 
#  total observation is 242, which is equal to number of items in a target vector,
#  each observation has 64*64 (=4096) features.
X_ = X.reshape(242, 64, 64, 1)
# convert to 3 channels
X_ = np.repeat(X_, 3, axis=3)
# apply preprocessing data, then change the ready-to-fit the model
X_ready = preprocess_input(X_)
X_ready.shape

(242, 64, 64, 3)

In [5]:
# train and test split
X_train, X_test, y_train, y_test = train_test_split(X_ready, y, test_size=0.25, random_state=0)

print('Training: ')
print(f'X: {X_train.shape}, y: {y_train.shape}')
print('Test: ')
print(f'X: {X_test.shape}, y: {y_test.shape}')

Training: 
X: (181, 64, 64, 3), y: (181,)
Test: 
X: (61, 64, 64, 3), y: (61,)


In [6]:
# loading VGG16
base_model = VGG16(weights='imagenet', include_top=False, input_tensor=Input(shape=(64, 64, 3)))

# Freeze the layers of VGG16
for layer in base_model.layers:
    layer.trainable = False

# show summary
base_model.summary()



Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 64, 64, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 64, 64, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 32, 32, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 32, 32, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 32, 32, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 16, 16, 128)       0   

In [7]:
# create new model
model = Sequential()
# add VGG16 model
model.add(base_model)

# new layers for binary classification
model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dense(4096, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# model summary
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 2, 2, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 4096)              8392704   
                                                                 
 dense_1 (Dense)             (None, 4096)              16781312  
                                                                 
 dense_2 (Dense)             (None, 1)                 4097      
                                                                 
Total params: 39892801 (152.18 MB)
Trainable params: 25178113 (96.05 MB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


In [8]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10


6/6 [==============================] - 7s 900ms/step - loss: 99.6410 - accuracy: 0.5304 - val_loss: 16.3724 - val_accuracy: 0.5246
Epoch 2/10
6/6 [==============================] - 5s 774ms/step - loss: 5.5025 - accuracy: 0.5249 - val_loss: 0.9710 - val_accuracy: 0.6721
Epoch 3/10
6/6 [==============================] - 6s 971ms/step - loss: 1.3504 - accuracy: 0.6740 - val_loss: 1.4596 - val_accuracy: 0.6066
Epoch 4/10
6/6 [==============================] - 5s 867ms/step - loss: 0.6051 - accuracy: 0.7293 - val_loss: 0.8991 - val_accuracy: 0.7213
Epoch 5/10
6/6 [==============================] - 5s 903ms/step - loss: 0.4170 - accuracy: 0.8011 - val_loss: 0.8513 - val_accuracy: 0.5574
Epoch 6/10
6/6 [==============================] - 6s 948ms/step - loss: 0.2973 - accuracy: 0.8729 - val_loss: 0.6811 - val_accuracy: 0.7541
Epoch 7/10
6/6 [==============================] - 5s 847ms/step - loss: 0.2298 - accuracy: 0.9061 - val_loss: 0.7992 - val_accuracy: 0.7377
Epoch 8/10
6/6 [